You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the current state of Artificial Intelligence (AI)
- Introduction to the latest trends and key players in the AI industry
- Preview of what readers can expect to learn from the article

Key Points:
1. Latest Trends in Artificial Intelligence:
   - Machine learning advancements
   - Natural language processing developments
   - AI in healthcare and finance industries
   - Ethical c

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has rapidly evolved in recent years, shaping industries and transforming the way we live and work. From machine learning advancements to ethical considerations in technology development, the AI landscape is constantly changing. In this article, we will explore the latest trends, key players, and noteworthy news in the AI industry, providing insights for tech enthusiasts, professionals, and business leaders interested in AI technology.

## Latest Trends in Artificial Intelligence
One of the most significant trends in AI is the continuous advancements in machine learning. Algorithms are becoming more sophisticated, allowing AI systems to learn and improve from data without explicit programming. Natural language processing (NLP) is also a key focus, with AI systems now able to understand and generate human language with increasing ac

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has rapidly evolved in recent years, shaping industries and transforming the way we live and work. From machine learning advancements to ethical considerations in technology development, the AI landscape is constantly changing. In this article, we will explore the latest trends, key players, and noteworthy news in the AI industry, providing insights for tech enthusiasts, professionals, and business leaders interested in AI technology.

## Latest Trends in Artificial Intelligence
One of the most significant trends in AI is the continuous advancements in machine learning. Algorithms are becoming more sophisticated, allowing AI systems to learn and improve from data without explicit programming. Natural language processing (NLP) is also a key focus, with AI systems now able to understand and generate human language with increasing accuracy. In addition, AI applications in healthcare and finance are revolutionizing these industries, from personalized medicine to algorithmic trading. However, as AI technologies become more powerful, ethical considerations surrounding privacy, bias, and accountability become increasingly important.

## Key Players in the AI Industry
Several companies are at the forefront of AI innovation, shaping the industry with their cutting-edge technologies. Google DeepMind, known for its breakthroughs in AI research, is pushing the boundaries of machine learning and AI applications. IBM Watson is another key player, offering AI-powered solutions for businesses across various sectors. Amazon Web Services and Microsoft Azure provide cloud-based AI services, enabling organizations to harness the power of AI without heavy investment in infrastructure. OpenAI, a nonprofit AI research organization, focuses on developing safe and beneficial AI for the world.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI research have led to exciting developments in areas such as computer vision, autonomous vehicles, and natural language understanding. AI applications continue to expand into sectors like education, logistics, and cybersecurity, driving efficiency and innovation. However, the impact of AI on the job market and society raises concerns about job displacement and ethical dilemmas. Regulatory updates in the AI industry aim to address these challenges, ensuring responsible AI development and deployment.

## Conclusion
As AI technology continues to advance, staying informed about the latest trends and key players in the industry is essential for tech enthusiasts, professionals, and business leaders. By understanding the implications of AI on society and economy, we can navigate the evolving landscape of AI with awareness and responsibility. Subscribe to our newsletter for regular updates and insights on AI technology, and join the conversation on the future of Artificial Intelligence.

Remember, the future of AI is in our hands, let's shape it wisely.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Meditation"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Meditation.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document:

Title: The Power of Meditation: A Comprehensive Guide to Finding Inner Peace

Outline:
I. Introduction
- Definition of meditation
- Importance of meditation in modern-day life
- Brief overview of the benefits of meditation

II. Key Points
A. Different types of meditation practices
1. Mindfulness meditation
2. Transcendental meditation
3. Loving-kindness meditation
B. Steps to start a meditation practice
1. Finding a quiet space
2. Setting a time frame
3. Focusing on breathing or a mantra
C. Benefit

I now can give a great answer

Final Answer:
# The Power of Meditation: A Comprehensive Guide to Finding Inner Peace

## Introduction

In today's fast-paced world, the practice of meditation has gained significant popularity as a means to find inner peace and improve overall well-being. Meditation can be defined as a technique that involves focusing the mind on a particular object, thought, or activity to achieve a mentally clear and emotionally calm state. With the increasing stress and anxiety in modern-day life, the importance of meditation has become more evident than ever. This comprehensive guide aims to explore the various types of meditation practices, the steps to start a meditation practice, and the numerous benefits that meditation can offer.

## Key Points

### Different types of meditation practices
1. **Mindfulness Meditation**: This form of meditation involves being fully present in the moment and aware of one's thoughts and feelings without judgment. It focuses on culti

In [14]:
Markdown(result)

# The Power of Meditation: A Comprehensive Guide to Finding Inner Peace

## Introduction

In today's fast-paced world, the practice of meditation has gained significant popularity as a means to find inner peace and improve overall well-being. Meditation can be defined as a technique that involves focusing the mind on a particular object, thought, or activity to achieve a mentally clear and emotionally calm state. With the increasing stress and anxiety in modern-day life, the importance of meditation has become more evident than ever. This comprehensive guide aims to explore the various types of meditation practices, the steps to start a meditation practice, and the numerous benefits that meditation can offer.

## Key Points

### Different types of meditation practices
1. **Mindfulness Meditation**: This form of meditation involves being fully present in the moment and aware of one's thoughts and feelings without judgment. It focuses on cultivating a heightened sense of awareness and acceptance.
2. **Transcendental Meditation**: A popular form of meditation that involves silently repeating a mantra to transcend ordinary thinking and reach a state of pure awareness and consciousness.
3. **Loving-Kindness Meditation**: Also known as Metta meditation, this practice involves cultivating feelings of love, compassion, and goodwill towards oneself and others.

### Steps to start a meditation practice
1. **Finding a Quiet Space**: It is essential to find a peaceful and comfortable space where you can practice meditation without distractions.
2. **Setting a Time Frame**: Establish a regular meditation routine by setting aside a specific time each day to dedicate to your practice.
3. **Focusing on Breathing or a Mantra**: Choose a focal point for your meditation, such as your breath or a mantra, to anchor your attention and quiet the mind.

### Benefits of meditation
1. **Reduced Stress and Anxiety**: By calming the mind and promoting relaxation, meditation can help reduce stress levels and alleviate symptoms of anxiety.
2. **Improved Focus and Concentration**: Regular meditation practice can enhance cognitive function, improve focus, and increase productivity.
3. **Enhanced Emotional Well-being**: Meditation can foster emotional resilience, promote self-awareness, and cultivate a sense of inner peace and contentment.

## Target Audience Analysis

The target audience for this guide includes individuals seeking stress relief, professionals looking to enhance productivity, students dealing with academic pressure, and anyone interested in personal growth and self-discovery through meditation. Whether you are a beginner looking to start a meditation practice or someone seeking to deepen your existing practice, this guide aims to provide valuable insights and resources to support your journey towards inner peace and well-being.

## Conclusion

In conclusion, the practice of meditation offers a powerful tool for finding inner peace, reducing stress, and improving overall well-being. By exploring different types of meditation practices, understanding the steps to start a meditation practice, and recognizing the benefits that meditation can bring, individuals can embark on a transformative journey towards self-discovery and personal growth. With the right guidance and commitment, meditation can become a valuable daily practice that nurtures the mind, body, and spirit.

Remember, the key to a successful meditation practice lies in consistency, patience, and a willingness to explore and embrace the present moment with openness and curiosity. Start your meditation journey today and experience the profound benefits that this ancient practice can offer.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).